## MSDS 492- Analysis of Financial Markets
## Assignment 3- GARCH Model Analysis 
## David Van Dyke

## Data Loading from Yfinance

In [2]:
"""
Download historical market data for SPY, GLD, CL=F, PSX, VLO, MPC,
and EUR to USD exchange rate (EURUSD=X)
and save each to a separate sheet in a single Excel file named
'GARCH model.xlsx'.

Enhancements:
- Uses SPY, GLD, CL=F, PSX, VLO, MPC, EURUSD=X
- Keeps yfinance's grouped (non-flattened) columns
- Normalizes tz on the index and labels the index as 'Date'
- Includes 'Adj Close' if available; if not, creates it by copying 'Close'
- Writes sheets with index=True (index as first column)
- No dividends/splits handling

"""

import os
from datetime import datetime
from typing import Tuple, Dict

import pandas as pd
import yfinance as yf

# -----------------------------
# Configuration
# -----------------------------

# Updated tickers to include EUR->USD exchange rate
TICKERS = ["SPY", "GLD", "CL=F", "PSX", "VLO", "MPC", "EURUSD=X"]

OUTPUT_XLSX = "GARCH model.xlsx"

INTERVAL = "1d"  # "1d", "1wk", "1mo"

# Updated date range: 1/1/2019 → today
START_DATE = datetime(2019, 1, 1)
END_DATE = datetime.today()

# Optional: friendlier sheet names (Excel sheet names are limited to 31 chars)
SHEET_NAME_MAP: Dict[str, str] = {
    "EURUSD=X": "EUR_USD",
}

# -----------------------------
# Helpers
# -----------------------------

def safe_sheet_name(name: str) -> str:
    """
    Excel sheet names are limited to 31 chars and cannot contain: : \ / ? * [ ]
    Trim and replace invalid characters to keep it safe.
    """
    invalid = [":", "\\", "/", "?", "*", "[", "]"]
    for ch in invalid:
        name = name.replace(ch, "-")
    return name[:31]


def ensure_adj_close(df: pd.DataFrame) -> Tuple[pd.DataFrame, bool]:
    """
    Ensure an 'Adj Close' column (or sub-column) exists.
    - If present, returns df unchanged and flag=False.
    - If absent, create it by copying 'Close' and return flag=True.

    Supports:
      • Single-level columns (e.g., 'Open', 'High', ..., 'Adj Close')
      • MultiIndex columns (yfinance layout with group_by='column')
    """
    created = False
    cols = df.columns

    # Case A: Single-level columns
    if isinstance(cols, pd.Index):
        if "Adj Close" in cols:
            return df, created
        if "Close" in cols:
            df = df.copy()
            df["Adj Close"] = df["Close"]
            created = True
        return df, created

    # Case B: MultiIndex columns
    if isinstance(cols, pd.MultiIndex):
        level0 = cols.get_level_values(0)

        if "Adj Close" in level0:
            return df, created

        if "Close" in level0:
            df = df.copy()
            sub_levels = cols[level0 == "Close"]
            new_cols = [("Adj Close", sub) for _, sub in sub_levels]
            copied = df.loc[:, sub_levels].copy()
            copied.columns = pd.MultiIndex.from_tuples(new_cols, names=df.columns.names)
            df = pd.concat([df, copied], axis=1)
            df = df.reindex(sorted(df.columns, key=lambda t: (t[0], str(t[1]))), axis=1)
            created = True
        return df, created

    return df, created


# -----------------------------
# yfinance wrapper
# -----------------------------

def download_ticker_df(
    ticker: str,
    start: datetime,
    end: datetime,
    interval: str = "1d"
) -> pd.DataFrame:
    """
    Download price/volume/OHLCV data for a single ticker.
    - Keeps yfinance's grouped, non-flattened columns (group_by='column').
    - Ensures tz-naive datetime index for consistent Excel output.
    - Ensures 'Adj Close' exists (creates from Close if provider doesn't supply it).
    """
    df = yf.download(
        tickers=ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        group_by="column",
        threads=True,
        progress=False,
    )

    if df is None or df.empty:
        return pd.DataFrame()

    idx_tz = getattr(df.index, "tz", None)
    if idx_tz is not None:
        df.index = df.index.tz_convert("UTC").tz_localize(None)

    df.index.name = "Date"
    df, _ = ensure_adj_close(df)

    return df


# -----------------------------
# Main workflow
# -----------------------------

def main():
    out_path = os.path.abspath(OUTPUT_XLSX)

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        for ticker in TICKERS:
            print(f"Downloading {ticker} from {START_DATE.date()} to {END_DATE.date()} ...")
            prices_df = download_ticker_df(ticker, START_DATE, END_DATE, INTERVAL)

            # Prefer friendly names when provided
            raw_sheet_name = SHEET_NAME_MAP.get(ticker, ticker)
            sheet_name = safe_sheet_name(raw_sheet_name)

            if prices_df.empty:
                pd.DataFrame({
                    "Info": [f"No price data returned for {ticker} in the requested window."]
                }).to_excel(writer, sheet_name=sheet_name, index=True)
            else:
                prices_df.to_excel(writer, sheet_name=sheet_name, index=True)

    print(f"✅ Excel file saved: {out_path}")


if __name__ == "__main__":
    main()

✅ Excel file saved: /mnt/batch/tasks/shared/LS_root/mounts/clusters/dvandyk1/code/Users/dvandyk/NW Data Science/MSDS 492 Analysis of Financial Markets/Assignment 3/GARCH model.xlsx


## Read yfinance Excel Workbook Into Pandas Dataframe

In [1]:
import pandas as pd
from typing import Dict, Optional, Tuple, Union

def _concat_two_levels_to_str(level0: Optional[object], level1: Optional[object], sep: str) -> str:
    """
    Helper to combine two header cells into a single string, handling None/NaN.
    Converts values to strings, strips whitespace, and drops blank parts.
    """
    parts = []
    for v in (level0, level1):
        if v is None:
            continue
        s = str(v).strip()
        if s != "" and s.lower() != "nan":
            parts.append(s)
    return sep.join(parts) if parts else ""


def _flatten_columns(cols: Union[pd.Index, pd.MultiIndex], sep: str = "|") -> pd.Index:
    """
    Flatten columns for easier downstream use.
    - If MultiIndex: join (level0, level1) as "level0|level1"
    - If Index: return as-is
    """
    if isinstance(cols, pd.MultiIndex):
        flat = []
        for a, b in cols.to_list():
            flat.append(_concat_two_levels_to_str(a, b, sep=sep))
        return pd.Index(flat)
    return cols


def read_market_prices_xlsx(
    path: str,
    parse_dates: bool = True,
    sep: str = "|",
    numeric_cleanup: bool = True
) -> Dict[str, pd.DataFrame]:
    """
    Reads the price sheets created by the download script (GARCH model.xlsx).
    Works with sheets written using DataFrame.to_excel(index=True) where:
      - The first column is the Date index
      - Columns may be single-level or MultiIndex (yfinance grouped layout)

    Returns a dict of DataFrames keyed by sheet name.

    Parameters
    ----------
    path : str
        Path to the Excel file.
    parse_dates : bool
        Convert first column to datetime index if True.
    sep : str
        Separator for flattening MultiIndex headers.
    numeric_cleanup : bool
        Strip formatting characters before numeric conversion.

    Returns
    -------
    Dict[str, pd.DataFrame]
        Mapping sheet_name -> cleaned DataFrame.
    """
    xls = pd.ExcelFile(path, engine="openpyxl")
    out: Dict[str, pd.DataFrame] = {}

    for sheet_name in xls.sheet_names:
        # Try reading as MultiIndex header first (common with yfinance "group_by='column'")
        df = None
        try:
            df = pd.read_excel(
                path,
                sheet_name=sheet_name,
                header=[0, 1],
                index_col=0,
                engine="openpyxl"
            )
            # If the second header row is effectively empty/NaN, this can produce odd columns.
            # We'll detect that and re-read as single header.
            if isinstance(df.columns, pd.MultiIndex):
                # If all second-level entries are NaN/blank, treat as single-level
                lvl1 = df.columns.get_level_values(1)
                if all((str(v).strip().lower() in ("nan", "") for v in lvl1)):
                    raise ValueError("Second header level empty; re-reading as single header.")
        except Exception:
            # Fallback: single header
            df = pd.read_excel(
                path,
                sheet_name=sheet_name,
                header=0,
                index_col=0,
                engine="openpyxl"
            )

        # If empty or just an info message sheet, keep but cleaned
        if df is None or df.empty:
            out[sheet_name] = pd.DataFrame()
            continue

        # Normalize index to datetime if requested
        if parse_dates:
            df.index = pd.to_datetime(df.index, errors="coerce")
            df = df[~df.index.isna()]

        df.index.name = "Date"

        # Drop all-NaN columns
        df = df.dropna(axis=1, how="all")

        # Flatten columns (MultiIndex -> strings)
        df.columns = _flatten_columns(df.columns, sep=sep)

        # Ensure unique column names
        if len(df.columns) != len(set(df.columns)):
            seen = {}
            newcols = []
            for c in df.columns:
                if c not in seen:
                    seen[c] = 1
                    newcols.append(c)
                else:
                    seen[c] += 1
                    newcols.append(f"{c}{sep}{seen[c]}")
            df.columns = newcols

        # Clean numeric formatting
        if numeric_cleanup:
            # Work column-wise for object/string columns
            for c in df.columns:
                if df[c].dtype == "object":
                    df[c] = df[c].astype(str).str.strip()
                    df[c] = df[c].replace({"": None, "nan": None, "NaN": None})
                    df[c] = df[c].str.replace(",", "", regex=False)
                    df[c] = df[c].str.replace("%", "", regex=False)

        # Convert everything to numeric where possible
        df = df.apply(pd.to_numeric, errors="coerce")

        out[sheet_name] = df

    return out


# ------------------------------
# Example usage
# ------------------------------
if __name__ == "__main__":

    # Read the Excel produced by your GARCH data pull
    xlsx_path = "GARCH model.xlsx"

    # Load all sheets
    frames = read_market_prices_xlsx(
        xlsx_path,
        sep="|",
        numeric_cleanup=True,
        parse_dates=True
    )

    # Create a DataFrame per ticker (matching sheet names in the Excel file)
    spy_df = frames.get("SPY")
    gld_df = frames.get("GLD")
    clf_df = frames.get("CL=F")
    psx_df = frames.get("PSX")
    vlo_df = frames.get("VLO")
    mpc_df = frames.get("MPC")
    eurusd_df = frames.get("EUR_USD")  

## EDA on GARCH Model Data

In [2]:
# -*- coding: utf-8 -*-
"""
EDA for GARCH model inputs using the 'GARCH model.xlsx' file produced by the
download script (with yfinance grouped columns) and the updated reader that
returns cleaned DataFrames per sheet.

Assets (example):
- SPY, GLD, CL=F, PSX, VLO, MPC
- FX: EURUSD=X saved as sheet "EUR_USD"
"""

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from typing import Dict, List, Optional, Union

# ----------------------------
# Load your data (updated inline reader for the NEW Excel layout)
# ----------------------------
def _concat_two_levels_to_str(level0: Optional[object], level1: Optional[object], sep: str) -> str:
    """
    Helper to combine two header cells into a single string, handling None/NaN.
    Converts values to strings, strips whitespace, and drops blank parts.
    """
    parts = []
    for v in (level0, level1):
        if v is None:
            continue
        s = str(v).strip()
        if s != "" and s.lower() != "nan":
            parts.append(s)
    return sep.join(parts) if parts else ""


def _flatten_columns(cols: Union[pd.Index, pd.MultiIndex], sep: str = "|") -> pd.Index:
    """
    Flatten columns for easier downstream use.
    - If MultiIndex: join (level0, level1) as "level0|level1"
    - If Index: return as-is
    """
    if isinstance(cols, pd.MultiIndex):
        flat = []
        for a, b in cols.to_list():
            flat.append(_concat_two_levels_to_str(a, b, sep=sep))
        return pd.Index(flat)
    return cols


def read_market_prices_xlsx(
    path: str,
    parse_dates: bool = True,
    sep: str = "|",
    numeric_cleanup: bool = True
) -> Dict[str, pd.DataFrame]:
    """
    Reads the price sheets created by the download script (GARCH model.xlsx).
    Works with sheets written using DataFrame.to_excel(index=True) where:
      - The first column is the Date index
      - Columns may be single-level or MultiIndex (yfinance grouped layout)

    Returns a dict of DataFrames keyed by sheet name.
    """
    xls = pd.ExcelFile(path, engine="openpyxl")
    out: Dict[str, pd.DataFrame] = {}

    for sheet_name in xls.sheet_names:
        # Try MultiIndex header first (typical yfinance "group_by='column'")
        try:
            df = pd.read_excel(
                path,
                sheet_name=sheet_name,
                header=[0, 1],
                index_col=0,
                engine="openpyxl"
            )
            # If second header level is effectively empty, re-read as single header
            if isinstance(df.columns, pd.MultiIndex):
                lvl1 = df.columns.get_level_values(1)
                if all((str(v).strip().lower() in ("nan", "") for v in lvl1)):
                    raise ValueError("Second header level empty; re-reading as single header.")
        except Exception:
            df = pd.read_excel(
                path,
                sheet_name=sheet_name,
                header=0,
                index_col=0,
                engine="openpyxl"
            )

        if df is None or df.empty:
            out[sheet_name] = pd.DataFrame()
            continue

        # Parse Date index
        if parse_dates:
            df.index = pd.to_datetime(df.index, errors="coerce")
            df = df[~df.index.isna()]
        df.index.name = "Date"

        # Drop all-NaN columns
        df = df.dropna(axis=1, how="all")

        # Flatten columns (MultiIndex -> strings)
        df.columns = _flatten_columns(df.columns, sep=sep)

        # Ensure unique column names
        if len(df.columns) != len(set(df.columns)):
            seen = {}
            newcols = []
            for c in df.columns:
                if c not in seen:
                    seen[c] = 1
                    newcols.append(c)
                else:
                    seen[c] += 1
                    newcols.append(f"{c}{sep}{seen[c]}")
            df.columns = newcols

        # Clean numeric formatting
        if numeric_cleanup:
            for c in df.columns:
                if df[c].dtype == "object":
                    df[c] = df[c].astype(str).str.strip()
                    df[c] = df[c].replace({"": None, "nan": None, "NaN": None})
                    df[c] = df[c].str.replace(",", "", regex=False)
                    df[c] = df[c].str.replace("%", "", regex=False)

        # Convert to numeric
        df = df.apply(pd.to_numeric, errors="coerce")

        out[sheet_name] = df

    return out


# ----------------------------
# Plot styling
# ----------------------------
plt.rcParams["figure.figsize"] = (11, 7)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.autolayout"] = True
sns.set_style("whitegrid")


# ----------------------------
# Helpers (updated/complete)
# ----------------------------
def _select_numeric_cols(df: pd.DataFrame) -> pd.DataFrame:
    return df.select_dtypes(include=[np.number]).copy()


def _infer_close_col(df: pd.DataFrame) -> Optional[str]:
    """
    Works with flattened columns like:
      - 'Adj Close|SPY', 'Close|SPY', etc.
    Also works if columns are single-level like 'Adj Close' or 'Close'.
    Preference order:
      1) Adj Close
      2) Close
      3) anything containing 'close'
    """
    cols_lower = {str(c).lower(): c for c in df.columns}

    # Prefer exact / contains patterns
    adj = next((cols_lower[k] for k in cols_lower if "adj close" in k), None)
    close_exact = next((cols_lower[k] for k in cols_lower if k == "close"), None)
    close_any = next((cols_lower[k] for k in cols_lower if "close" in k), None)

    return next((c for c in (adj, close_exact, close_any) if c is not None), None)


def _daily_returns(series: pd.Series) -> pd.Series:
    """
    Log returns: ln(P_t) - ln(P_{t-1})
    """
    s = pd.to_numeric(series, errors="coerce").astype(float)
    s = s.replace([0, np.inf, -np.inf], np.nan).dropna()
    return np.log(s).diff()


def _safe_title(name: str) -> str:
    return str(name).strip() if name else "Unknown"


def _inject_index_as_column(df: pd.DataFrame, name: str = "Variable") -> pd.DataFrame:
    idx_name = df.index.name if df.index.name not in [None, ""] else name
    out = df.reset_index()
    if out.columns[0] != idx_name:
        out = out.rename(columns={out.columns[0]: idx_name})
    return out


def _table_figure_from_dataframe(df: pd.DataFrame, title: str, font_size: int = 9) -> plt.Figure:
    fig_height = min(7, 1 + 0.35 * (len(df) + 2))
    fig, ax = plt.subplots(figsize=(11, fig_height))
    ax.axis("off")
    ax.set_title(title, fontsize=14, pad=12, loc="left")
    tbl = ax.table(
        cellText=df.round(4).values.tolist(),
        colLabels=list(df.columns),
        loc="center",
        cellLoc="right",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(font_size)
    tbl.scale(1, 1.25)
    return fig


def _corr_heatmap(df: pd.DataFrame, title: str) -> plt.Figure:
    num = _select_numeric_cols(df)
    if num.empty:
        msg = pd.DataFrame({"Message": ["No numeric columns found."]})
        msg = _inject_index_as_column(msg, "Info")
        return _table_figure_from_dataframe(msg, title)

    corr = num.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr, cmap="coolwarm", center=0, ax=ax)
    ax.set_title(title)
    return fig


def _rolling_volatility(series: pd.Series, window: int = 30) -> pd.Series:
    rets = _daily_returns(series).dropna()
    return rets.rolling(window).std() * math.sqrt(252)


def _schema_and_missing_table(df: pd.DataFrame) -> pd.DataFrame:
    """
    Summarize column schema + missingness.
    """
    rows = []
    n = len(df)
    for c in df.columns:
        s = df[c]
        miss = int(s.isna().sum())
        nonmiss = int(s.notna().sum())
        rows.append({
            "dtype": str(s.dtype),
            "count": nonmiss,
            "missing": miss,
            "missing_pct": (miss / n * 100.0) if n else np.nan,
        })
    out = pd.DataFrame(rows, index=df.columns)
    out.index.name = "Variable"
    return out.sort_values(["missing", "dtype"], ascending=[False, True])


def _numeric_quality_and_ranges(df: pd.DataFrame) -> pd.DataFrame:
    """
    For numeric columns: counts, missing, min/max, and basic data quality flags.
    """
    num = _select_numeric_cols(df)
    if num.empty:
        out = pd.DataFrame({"Message": ["No numeric columns found."]}, index=["Info"])
        out.index.name = "Variable"
        return out

    n = len(num)
    rows = []
    for c in num.columns:
        s = num[c].astype(float)
        miss = int(s.isna().sum())
        nonmiss = int(s.notna().sum())
        zeros = int((s == 0).sum(skipna=True))
        neg = int((s < 0).sum(skipna=True))
        infs = int(np.isinf(s.to_numpy(dtype=float, na_value=np.nan)).sum())  # mostly 0 after coercion
        rows.append({
            "count": nonmiss,
            "missing": miss,
            "missing_pct": (miss / n * 100.0) if n else np.nan,
            "zeros": zeros,
            "negatives": neg,
            "infs": infs,
            "min": float(np.nanmin(s.to_numpy())) if nonmiss else np.nan,
            "max": float(np.nanmax(s.to_numpy())) if nonmiss else np.nan,
            "mean": float(np.nanmean(s.to_numpy())) if nonmiss else np.nan,
            "std": float(np.nanstd(s.to_numpy(), ddof=1)) if nonmiss > 1 else np.nan,
        })

    out = pd.DataFrame(rows, index=num.columns)
    out.index.name = "Variable"
    return out.sort_values(["missing", "std"], ascending=[False, False])


# ----------------------------
# Single-ticker EDA
# ----------------------------
def eda_single_ticker(name: str, df: pd.DataFrame) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    ticker = _safe_title(name)
    df = df.copy().sort_index()

    # Schema
    tab = _schema_and_missing_table(df)
    tab = _inject_index_as_column(tab, "Variable")
    figs.append(_table_figure_from_dataframe(tab, f"{ticker} • Schema & Missingness"))

    # Numeric quality
    tab2 = _numeric_quality_and_ranges(df)
    tab2 = _inject_index_as_column(tab2, "Variable")
    figs.append(_table_figure_from_dataframe(tab2, f"{ticker} • Numeric Quality & Ranges"))

    # Describe
    num = _select_numeric_cols(df)
    if not num.empty:
        desc = num.describe().T
        desc = _inject_index_as_column(desc, "Variable")
        figs.append(_table_figure_from_dataframe(desc, f"{ticker} • Describe"))
    else:
        msg = pd.DataFrame({"Message": ["No numeric columns."]})
        msg = _inject_index_as_column(msg)
        figs.append(_table_figure_from_dataframe(msg, f"{ticker} • Describe"))

    # Correlation
    figs.append(_corr_heatmap(df, f"{ticker} • Correlation"))

    # Prices + volatility + returns
    close_col = _infer_close_col(df)
    if close_col:
        price = pd.to_numeric(df[close_col], errors="coerce").astype(float)

        # Price plot
        fig, ax = plt.subplots()
        ax.plot(price.index, price.values)
        ax.set_title(f"{ticker} • Price ({close_col})")
        figs.append(fig)

        # Rolling vol plot
        fig, ax = plt.subplots()
        vol = _rolling_volatility(price)
        ax.plot(vol.index, vol.values)
        ax.set_title(f"{ticker} • 30D Volatility (ann.)")
        figs.append(fig)

        # Returns histogram
        fig, ax = plt.subplots()
        rets = _daily_returns(price).dropna()
        if not rets.empty:
            sns.histplot(rets, bins=50, kde=True, ax=ax)
            ax.set_title(f"{ticker} • Daily Log Returns")
        else:
            ax.text(0.1, 0.5, "Returns series is empty after cleaning.", fontsize=12)
            ax.set_title(f"{ticker} • Daily Log Returns")
        figs.append(fig)

    return figs


# ----------------------------
# Cross-ticker EDA
# ----------------------------
def eda_cross_ticker(frames: Dict[str, pd.DataFrame]) -> List[plt.Figure]:
    figs: List[plt.Figure] = []

    price_map = {}
    for name, df in frames.items():
        col = _infer_close_col(df)
        if col:
            price_map[name] = pd.to_numeric(df[col], errors="coerce").astype(float)

    if price_map:
        prices = pd.DataFrame(price_map).dropna(how="all")

        # Overlay prices
        fig, ax = plt.subplots()
        for name in prices.columns:
            ax.plot(prices.index, prices[name], label=name)
        ax.set_title("Cross-Ticker Prices (Close/Adj Close)")
        ax.legend()
        figs.append(fig)

        # Return correlation
        rets = prices.apply(lambda s: _daily_returns(s)).dropna(how="all")
        if not rets.empty:
            corr = rets.corr()
            fig, ax = plt.subplots()
            sns.heatmap(corr, annot=True, cmap="viridis", ax=ax)
            ax.set_title("Cross-Ticker Return Correlation (log returns)")
            figs.append(fig)

    return figs


# ----------------------------
# Report Orchestration
# ----------------------------
def build_garch_report(
    frames: Dict[str, pd.DataFrame],
    pdf_title: str = "EDA GARCH model report.pdf",
    show_plots: bool = False,
):
    figs: List[plt.Figure] = []
    ordered = sorted(frames.items(), key=lambda kv: kv[0])

    for name, df in ordered:
        figs.extend(eda_single_ticker(name, df))

    figs.extend(eda_cross_ticker(frames))

    with PdfPages(pdf_title) as pdf:
        cover, ax = plt.subplots(figsize=(11, 8))
        ax.axis("off")
        ax.text(0, 0.95, "EDA GARCH Model Report", fontsize=24, weight="bold")
        ax.text(0, 0.88, f"Sheets: {', '.join(frames.keys())}", fontsize=12)
        ax.text(0, 0.82, "Generated automatically.", fontsize=12)
        pdf.savefig(cover)
        plt.close(cover)

        for fig in figs:
            pdf.savefig(fig)
            plt.close(fig)

    print(f"✅ PDF saved: {pdf_title}")

    if show_plots:
        for fig in figs:
            plt.show()


# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    xlsx_path = "GARCH model.xlsx"

    # Read using the NEW reader that matches the new workbook structure
    frames = read_market_prices_xlsx(
        xlsx_path,
        sep="|",
        numeric_cleanup=True,
        parse_dates=True
    )

    # Target assets: original set + FX sheet
    # FX sheet name is "EUR_USD" if you used SHEET_NAME_MAP in the download script,
    # otherwise it may be "EURUSD=X". We accept either.
    target = {"SPY", "GLD", "CL=F", "PSX", "VLO", "MPC", "EUR_USD", "EURUSD=X"}
    frames = {k: v for k, v in frames.items() if k in target and v is not None and not v.empty}

    if not frames:
        raise RuntimeError("No valid sheets found in GARCH model.xlsx for the target set.")

    build_garch_report(frames)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_11276/2035653122.py:342: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


✅ PDF saved: EDA GARCH model report.pdf


## Engineered Features

In [3]:
import pandas as pd
import numpy as np
from typing import Dict, Optional

# ================================================================
# Helpers
# ================================================================
def _find_col(df: pd.DataFrame, keyword: str) -> Optional[str]:
    """
    Find the first column whose name contains `keyword` (case-insensitive).
    """
    keyword = keyword.lower()
    for c in df.columns:
        if keyword in str(c).lower():
            return c
    return None


def _safe_numeric(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s, errors="coerce").astype(float)


# ================================================================
# Feature Engineering for GARCH-ready series
# ================================================================
def engineer_features_for_ticker(
    df: pd.DataFrame,
    ticker: str,
    ewma_lambda: float = 0.94
) -> pd.DataFrame:
    """
    Adds engineered features for a given asset DataFrame.

    Compatible with the NEW flattened yfinance layout:
        e.g. 'Adj Close|SPY', 'Close|SPY', etc.

    Core GARCH-oriented features:
        - log_ret, ret_sq, ret_abs
        - lagged returns (1–3)
        - lagged squared returns (1–3)
        - EWMA variance (RiskMetrics)
        - rolling variance (20)

    Additional diagnostics / exploratory:
        - HL / OC spreads (if available)
        - SMA / EMA (5/10/20)
        - lagged prices (1–3)
        - autocorr snapshots
        - calendar seasonality
    """

    out = df.copy().sort_index()

    # ------------------------------------------------
    # Seasonal / calendar features
    # ------------------------------------------------
    out["month"]          = out.index.month
    out["day_of_week"]    = out.index.dayofweek
    out["quarter"]        = out.index.quarter
    out["year"]           = out.index.year
    out["is_month_end"]   = out.index.is_month_end.astype(int)
    out["is_month_start"] = out.index.is_month_start.astype(int)

    # ------------------------------------------------
    # Infer price-related columns (robust)
    # ------------------------------------------------
    adj_close_col = _find_col(out, "adj close")
    close_col     = _find_col(out, "close")
    high_col      = _find_col(out, "high")
    low_col       = _find_col(out, "low")
    open_col      = _find_col(out, "open")

    # Prefer Adj Close when available
    price_col = adj_close_col if adj_close_col is not None else close_col
    if price_col is None:
        raise ValueError(f"No Close or Adj Close column found for {ticker}")

    price = _safe_numeric(out[price_col])
    high  = _safe_numeric(out[high_col])  if high_col  else None
    low   = _safe_numeric(out[low_col])   if low_col   else None
    open_ = _safe_numeric(out[open_col])  if open_col  else None
    close = _safe_numeric(out[close_col]) if close_col else None

    # ------------------------------------------------
    # Spread features (only if data exists)
    # ------------------------------------------------
    out[f"HL_spread|{ticker}"] = (high - low) if (high is not None and low is not None) else np.nan
    out[f"OC_spread|{ticker}"] = (open_ - close) if (open_ is not None and close is not None) else np.nan

    # ------------------------------------------------
    # SMA / EMA (exploratory)
    # ------------------------------------------------
    for w in (5, 10, 20):
        out[f"SMA_{w}|{ticker}"] = price.rolling(w, min_periods=1).mean()
        out[f"EMA_{w}|{ticker}"] = price.ewm(span=w, adjust=False, min_periods=1).mean()

    # ------------------------------------------------
    # Lagged prices (exploratory)
    # ------------------------------------------------
    for k in (1, 2, 3):
        out[f"lag_price_{k}|{ticker}"] = price.shift(k)

    # ------------------------------------------------
    # Price autocorrelation snapshots (scalar, repeated)
    # ------------------------------------------------
    for k in (1, 2, 3):
        out[f"autocorr_price_{k}|{ticker}"] = price.corr(price.shift(k))

    # ------------------------------------------------
    # Log returns (CORE for GARCH)
    # ------------------------------------------------
    out[f"log_ret|{ticker}"] = np.log(price / price.shift(1))
    ret = out[f"log_ret|{ticker}"]

    out[f"ret_sq|{ticker}"]  = ret ** 2
    out[f"ret_abs|{ticker}"] = ret.abs()

    # ------------------------------------------------
    # Lagged returns (ARCH diagnostics)
    # ------------------------------------------------
    for k in (1, 2, 3):
        out[f"lag_ret_{k}|{ticker}"]    = ret.shift(k)
        out[f"lag_ret_sq_{k}|{ticker}"] = (ret ** 2).shift(k)

    # ------------------------------------------------
    # Variance proxies
    # ------------------------------------------------
    # RiskMetrics EWMA variance
    out[f"ewma_var|{ticker}"] = (
        ret.pow(2)
           .ewm(alpha=(1 - ewma_lambda), adjust=False)
           .mean()
    )

    # Rolling variance
    out[f"roll_var_20|{ticker}"] = ret.rolling(20, min_periods=5).var()

    # ------------------------------------------------
    # Return autocorrelation snapshots
    # ------------------------------------------------
    for k in (1, 2, 3):
        out[f"autocorr_ret_{k}|{ticker}"] = ret.corr(ret.shift(k))

    return out


# ================================================================
# Apply feature engineering across all assets
# ================================================================
def engineer_features(frames: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Applies feature engineering to each asset DataFrame.

    Assumes:
        frames keys = sheet names
        frames values = cleaned DataFrames from the NEW reader
    """
    engineered: Dict[str, pd.DataFrame] = {}

    for name, df in frames.items():
        if df is None or df.empty:
            continue
        engineered[name] = engineer_features_for_ticker(df, name)

    return engineered


# ================================================================
# Example usage
# ================================================================
if __name__ == "__main__":
    # Assumes you already ran:
    # frames = read_market_prices_xlsx("GARCH model.xlsx", parse_dates=True)

    engineered = engineer_features(frames)

    spy_df    = engineered.get("SPY")
    gld_df    = engineered.get("GLD")
    clf_df    = engineered.get("CL=F")
    psx_df    = engineered.get("PSX")
    vlo_df    = engineered.get("VLO")
    mpc_df    = engineered.get("MPC")
    eurusd_df = engineered.get("EUR_USD")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## EDA on Engineered Features

In [4]:
# -*- coding: utf-8 -*-
"""
Engineered Feature EDA Report for GARCH Model Inputs:
SPY, GLD, CL=F, PSX, VLO, MPC, EUR_USD

Runs directly on already-loaded DataFrames:

    spy_df, gld_df, clf_df, psx_df, vlo_df, mpc_df, EUR_USD

Does NOT read Excel files.
"""

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from typing import Dict, List, Optional, Any

# --------------------------------------
# Plot Styling
# --------------------------------------
plt.rcParams["figure.figsize"] = (11, 7)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.autolayout"] = True
sns.set_style("whitegrid")


# --------------------------------------
# Helpers
# --------------------------------------
def _safe_title(name: str) -> str:
    return str(name).strip() if name else "Unknown"

def _select_numeric_cols(df: pd.DataFrame) -> pd.DataFrame:
    return df.select_dtypes(include=[np.number]).copy()

def _infer_close_col(df: pd.DataFrame) -> Optional[str]:
    """
    Identify a 'Close-like' column such as:
      Adj Close|SPY
      Close|SPY
      close or adj close anywhere in the name
    """
    lower_map = {str(c).lower(): c for c in df.columns}

    # Priority 1: Adj Close
    for key, orig in lower_map.items():
        if "adj close" in key:
            return orig

    # Priority 2: exact "close"
    if "close" in lower_map:
        return lower_map["close"]

    # Priority 3: any containing "close"
    for key, orig in lower_map.items():
        if "close" in key:
            return orig

    return None

def _daily_returns(series: pd.Series) -> pd.Series:
    return np.log(series).diff()

def _inject_index_as_column(df: pd.DataFrame, name: str = "Variable") -> pd.DataFrame:
    idx_name = df.index.name if df.index.name not in [None, ""] else name
    out = df.reset_index()
    if out.columns[0] != idx_name:
        out = out.rename(columns={out.columns[0]: idx_name})
    return out

def _table_figure_from_dataframe(df: pd.DataFrame, title: str, font_size: int = 9) -> plt.Figure:
    fig_height = min(7, 1 + 0.35 * (len(df) + 2))
    fig, ax = plt.subplots(figsize=(11, fig_height))
    ax.axis("off")
    ax.set_title(title, fontsize=14, loc="left")

    tbl = ax.table(
        cellText=df.round(4).values.tolist(),
        colLabels=list(df.columns),
        loc="center",
        cellLoc="right"
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(font_size)
    tbl.scale(1, 1.25)
    return fig

def _corr_heatmap(df: pd.DataFrame, title: str) -> plt.Figure:
    num = _select_numeric_cols(df)
    if num.empty:
        msg = pd.DataFrame({"Message": ["No numeric columns found."]})
        msg = _inject_index_as_column(msg)
        return _table_figure_from_dataframe(msg, title)

    corr = num.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr, cmap="coolwarm", center=0, ax=ax)
    ax.set_title(title)
    return fig

def _rolling_volatility(series: pd.Series, window: int = 30) -> pd.Series:
    rets = np.log(series).diff().dropna()
    return rets.rolling(window).std() * math.sqrt(252)


# --------------------------------------
# Numeric EDA Tables
# --------------------------------------
def _schema_and_missing_table(df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    n = len(df)

    for col in df.columns:
        s = df[col]
        non_null = s.notna().sum()
        missing = s.isna().sum()

        try:
            unique = s.nunique(dropna=True)
        except:
            unique = np.nan

        sample = s.dropna().iloc[0] if non_null > 0 else None

        rows.append({
            "Column": col,
            "dtype": str(s.dtype),
            "non_null": non_null,
            "missing": missing,
            "missing_%": round(missing / n * 100, 2),
            "unique": unique,
            "sample": sample
        })

    return pd.DataFrame(rows).set_index("Column")

def _numeric_quality_and_ranges(df: pd.DataFrame) -> pd.DataFrame:
    num = _select_numeric_cols(df)
    if num.empty:
        return pd.DataFrame({"Message": ["No numeric columns found."]})

    rows = []
    num = num.replace([np.inf, -np.inf], np.nan)

    for col in num.columns:
        s = num[col]
        non_null = s.notna().sum()
        missing = s.isna().sum()

        zeros = (s == 0).sum() if non_null > 0 else 0
        negatives = (s < 0).sum() if non_null > 0 else 0

        stats = {
            "min": s.min(),
            "p01": s.quantile(0.01),
            "q1": s.quantile(0.25),
            "median": s.quantile(0.50),
            "mean": s.mean(),
            "q3": s.quantile(0.75),
            "p99": s.quantile(0.99),
            "max": s.max(),
            "std": s.std()
        }

        rows.append({
            "Column": col,
            "non_null": non_null,
            "missing": missing,
            "missing_%": round(missing / len(num) * 100, 2),
            "zeros": zeros,
            "negatives": negatives,
            **stats
        })

    return pd.DataFrame(rows).set_index("Column")


# --------------------------------------
# Per-Ticker EDA
# --------------------------------------
def eda_single_ticker(name: str, df: pd.DataFrame) -> List[plt.Figure]:
    figs = []
    df = df.copy().sort_index()

    # Schema
    schema = _schema_and_missing_table(df)
    schema = _inject_index_as_column(schema)
    figs.append(_table_figure_from_dataframe(schema, f"{name} • Schema & Missingness"))

    # Numeric Quality
    quality = _numeric_quality_and_ranges(df)
    quality = _inject_index_as_column(quality)
    figs.append(_table_figure_from_dataframe(quality, f"{name} • Numeric Quality & Ranges"))

    # Describe
    num = _select_numeric_cols(df)
    if not num.empty:
        desc = num.describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99]).T
        desc = _inject_index_as_column(desc)
        figs.append(_table_figure_from_dataframe(desc, f"{name} • Describe"))
    else:
        figs.append(_table_figure_from_dataframe(
            pd.DataFrame({"Message":["No numeric columns"]}),
            f"{name} • Describe"
        ))

    # Correlation
    figs.append(_corr_heatmap(df, f"{name} • Correlation"))

    # Price Features
    close_col = _infer_close_col(df)
    if close_col:
        price = df[close_col].astype(float)

        # Price Plot
        fig, ax = plt.subplots()
        ax.plot(price.index, price.values)
        ax.set_title(f"{name} • {close_col} Price")
        figs.append(fig)

        # Rolling Vol
        vol = _rolling_volatility(price)
        fig, ax = plt.subplots()
        ax.plot(vol.index, vol.values, color="darkorange")
        ax.set_title(f"{name} • 30-Day Rolling Volatility")
        figs.append(fig)

        # Return Distribution
        rets = np.log(price).diff().dropna()
        fig, ax = plt.subplots()
        sns.histplot(rets, bins=60, kde=True, color="teal")
        ax.set_title(f"{name} • Log Return Distribution")
        figs.append(fig)

    return figs


# --------------------------------------
# Cross-Ticker EDA
# --------------------------------------
def eda_cross_ticker(frames: Dict[str, pd.DataFrame]) -> List[plt.Figure]:
    figs = []
    price_map = {}

    for name, df in frames.items():
        col = _infer_close_col(df)
        if col:
            price_map[name] = df[col].astype(float)

    if price_map:
        prices = pd.DataFrame(price_map).sort_index()

        # Price Overlay
        fig, ax = plt.subplots()
        for name in prices.columns:
            ax.plot(prices.index, prices[name], lw=1.2, label=name)
        ax.legend()
        ax.set_title("Cross-Ticker • Close Prices")
        figs.append(fig)

        # Return Correlation
        rets = np.log(prices).diff().dropna()
        fig, ax = plt.subplots()
        sns.heatmap(rets.corr(), annot=True, cmap="viridis")
        ax.set_title("Cross-Ticker • Log Return Corr")
        figs.append(fig)

    return figs


# --------------------------------------
# Build PDF Report
# --------------------------------------
def build_engineered_feature_eda(
    frames: Dict[str, pd.DataFrame],
    pdf_title: str = "Engineered Feature EDA.pdf",
    show_plots: bool = False
):
    figs = []
    ordered = sorted(frames.items(), key=lambda x: x[0])

    for name, df in ordered:
        figs.extend(eda_single_ticker(name, df))

    figs.extend(eda_cross_ticker(frames))

    with PdfPages(pdf_title) as pdf:

        cover, ax = plt.subplots()
        ax.axis("off")
        ax.text(0.02, 0.92, "Engineered Feature EDA Report", fontsize=22, weight="bold")
        ax.text(0.02, 0.84, "Tickers: " + ", ".join(frames.keys()), fontsize=12)

        lines = [
            "• Schema & Missingness",
            "• Numeric Quality & Ranges",
            "• Numeric Overview",
            "• Correlation Heatmap",
            "• Price Trend & Volatility",
            "• Log Return Distribution",
            "• Cross-Ticker Price Overlay",
            "• Cross-Ticker Return Correlation"
        ]

        for i, l in enumerate(lines):
            ax.text(0.03, 0.75 - 0.05*i, l, fontsize=11)

        pdf.savefig(cover)
        plt.close(cover)

        for fig in figs:
            pdf.savefig(fig)
            plt.close(fig)

    print(f"📄 PDF saved: {pdf_title}")

    if show_plots:
        for fig in figs:
            plt.show()


# --------------------------------------
# Example Usage
# --------------------------------------
if __name__ == "__main__":

    # Already-loaded engineered DataFrames
    frames = {
        "SPY": spy_df,
        "GLD": gld_df,
        "CL=F": clf_df,
        "PSX": psx_df,
        "VLO": vlo_df,
        "MPC": mpc_df,
        "EUR_USD": eurusd_df
    }

    build_engineered_feature_eda(frames, pdf_title="Engineered Feature GARCH EDA.pdf")

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/tmp/ipykernel_11276/2448325007.py:234: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/pandas/core/internals/blocks.py:395: RuntimeWarning: invalid value encountered in log
  result = func(self.values, **kwargs)


📄 PDF saved: Engineered Feature GARCH EDA.pdf


## Verify Financial Market Stylized facts

In [6]:
# -*- coding: utf-8 -*-
"""
Multi-Asset Plotting Suite — Stylized Facts & Seasonality (Single-Ticker Frames)

Updates in this version:
- Uses your new frames/tickers:
    "SPY": spy_df,
    "GLD": gld_df,
    "CL=F": clf_df,
    "PSX": psx_df,
    "VLO": vlo_df,
    "MPC": mpc_df,
    "EUR_USD": eurusd_df
- More robust column detection for single-ticker frames:
    Works with either:
      "Adj Close|TICKER" / "Close|TICKER"   (engineered multi-asset naming)
    OR
      "Adj Close" / "Close"                (plain single-ticker naming)
    Same idea for "log_ret|TICKER" vs "log_ret".
- Regex feature detection now safely handles tickers like "CL=F" via re.escape().

"""

import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.backends.backend_pdf import PdfPages
from typing import Dict, List, Optional
from scipy import stats

# ----------------------------
# Plot styling
# ----------------------------
plt.rcParams["figure.figsize"] = (11, 7)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.autolayout"] = True
sns.set_style("whitegrid")

# ----------------------------
# Helpers
# ----------------------------
def _safe_title(name: str) -> str:
    return str(name).strip() if name else "Unknown"

def _table_figure_from_dataframe(df: pd.DataFrame, title: str, font_size: int = 9) -> plt.Figure:
    fig, ax = plt.subplots(figsize=(11, min(7, 1 + 0.35 * (len(df) + 2))))
    ax.axis("off")
    ax.set_title(title, fontsize=14, pad=12, loc="left")
    tbl = ax.table(
        cellText=df.round(4).values.tolist(),
        colLabels=[str(c) for c in df.columns],
        loc="center",
        cellLoc="right",
    )
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(font_size)
    tbl.scale(1, 1.2)
    return fig

def _numeric_clean(series: pd.Series) -> pd.Series:
    """Coerce to numeric and drop NaNs."""
    return pd.to_numeric(series, errors="coerce").dropna()

def _find_col_case_insensitive(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    """Return the first matching column from candidates (case-insensitive), else None."""
    low = {str(c).lower(): c for c in df.columns}
    for cand in candidates:
        hit = low.get(str(cand).lower())
        if hit is not None:
            return hit
    return None

def _adj_or_close_col(df: pd.DataFrame, ticker: str) -> Optional[str]:
    """
    Prefer adjusted close; support both:
      - "Adj Close|TICKER" / "Close|TICKER"
      - "Adj Close" / "Close"
    Also tries common alternates like "AdjClose", "Adj_Close", etc.
    """
    candidates = [
        f"Adj Close|{ticker}", f"Close|{ticker}",
        "Adj Close", "Close",
        f"AdjClose|{ticker}", f"Adj_Close|{ticker}", f"Adjusted Close|{ticker}",
        "AdjClose", "Adj_Close", "Adjusted Close",
    ]
    return _find_col_case_insensitive(df, candidates)

# ----------------------------
# Detect engineered feature columns
# ----------------------------
def _feature_cols_for_ticker(df: pd.DataFrame, ticker: str) -> Dict[str, List[str]]:
    """
    Supports both engineered multi-asset naming:
        HL_spread|TICKER, SMA_20|TICKER, lag_1|TICKER
    and plain single-ticker naming:
        HL_spread, SMA_20, lag_1
    """
    t = re.escape(ticker)

    # engineered patterns (with |TICKER)
    spreads_pat_t = re.compile(rf"^(HL_spread|OC_spread)\|{t}$")
    ma_pat_t      = re.compile(rf"^(SMA_\d+|EMA_\d+)\|{t}$")
    lag_pat_t     = re.compile(rf"^lag_\d+\|{t}$")

    # plain patterns (no suffix)
    spreads_pat   = re.compile(r"^(HL_spread|OC_spread)$")
    ma_pat        = re.compile(r"^(SMA_\d+|EMA_\d+)$")
    lag_pat       = re.compile(r"^lag_\d+$")

    cols = [str(c) for c in df.columns]
    spreads = [c for c in cols if spreads_pat_t.match(c) or spreads_pat.match(c)]
    ma      = [c for c in cols if ma_pat_t.match(c)      or ma_pat.match(c)]
    lags    = [c for c in cols if lag_pat_t.match(c)     or lag_pat.match(c)]

    return {"spreads": spreads, "ma": ma, "lags": lags}

# ----------------------------
# Return series aggregator
# ----------------------------
def _get_return_series(df: pd.DataFrame, ticker: str) -> Optional[pd.Series]:
    """
    Returns daily log returns from engineered column or derives from price.

    Supports both:
      - "log_ret|TICKER" or "log_ret"
      - and derivation from price column(s)
    """
    # engineered return col
    ret_col = _find_col_case_insensitive(df, [f"log_ret|{ticker}", "log_ret"])
    if ret_col is not None:
        s = _numeric_clean(df[ret_col]).sort_index()
        return s if not s.empty else None

    # derive from price
    price_col = _adj_or_close_col(df, ticker)
    if price_col:
        p = _numeric_clean(df[price_col]).sort_index()
        if not p.empty:
            return np.log(p / p.shift(1)).dropna()

    return None

# ----------------------------
# Core Plot Helpers (stylized facts)
# ----------------------------
def _plot_price_series(price: pd.Series, ticker: str) -> plt.Figure:
    fig, ax = plt.subplots()
    ax.plot(price.index, price.values, label="Adj/Close", color="royalblue", lw=1.6)
    ax.set_title(f"{ticker} • Price/Level")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price / Level")
    ax.legend(loc="best")
    return fig

def _plot_price_acf(price: pd.Series, ticker: str, max_lags: int = 20) -> plt.Figure:
    title = f"{ticker} • ACF • Price/Level"
    try:
        from statsmodels.graphics.tsaplots import plot_acf
        fig = plot_acf(price.dropna(), lags=max_lags)
        fig.axes[0].set_title(title)
        return fig
    except Exception:
        x = price.dropna()
        lags = list(range(1, max_lags + 1))
        values = [x.corr(x.shift(k)) for k in lags]
        fig, ax = plt.subplots()
        ax.bar(lags, values, color="slategray")
        ax.axhline(0, color="gray", lw=0.8)
        ax.set_title(f"{title} (manual)")
        ax.set_xlabel("Lag")
        ax.set_ylabel("Autocorrelation")
        return fig

def _plot_mas(price: pd.Series, ticker: str) -> plt.Figure:
    s = price.sort_index()
    ma_5  = s.rolling(window=5,  min_periods=5).mean()
    ma_10 = s.rolling(window=10, min_periods=10).mean()
    ma_20 = s.rolling(window=20, min_periods=20).mean()
    fig, ax = plt.subplots()
    ax.plot(s.index, s.values, color="royalblue", lw=1.0, alpha=0.6, label="Adj/Close")
    ax.plot(ma_5.index, ma_5.values, color="darkorange", lw=1.8, label="SMA 5")
    ax.plot(ma_10.index, ma_10.values, color="seagreen", lw=1.8, label="SMA 10")
    ax.plot(ma_20.index, ma_20.values, color="crimson", lw=1.8, label="SMA 20")
    ax.set_title(f"{ticker} • Moving Averages (5/10/20)")
    ax.set_xlabel("Date")
    ax.set_ylabel("Price / Level")
    ax.legend(loc="best")
    return fig

def _plot_returns_ts(r: pd.Series, ticker: str) -> plt.Figure:
    fig, ax = plt.subplots()
    ax.plot(r.index, r.values, color="darkorange", lw=1.2)
    ax.axhline(0.0, color="gray", lw=0.8)
    ax.set_title(f"{ticker} • Daily Log Returns")
    ax.set_xlabel("Date")
    ax.set_ylabel("Daily log return")
    return fig

def _plot_returns_hist_qq(r: pd.Series, ticker: str) -> List[plt.Figure]:
    """Histogram + KDE with stats box (mean/median/mode/std/skew/kurt) and a Q–Q plot."""
    figs: List[plt.Figure] = []

    # Histogram + KDE + stats box
    fig_hist, ax_hist = plt.subplots()
    sns.histplot(r, bins=60, kde=True, stat="density", color="teal", ax=ax_hist)
    ax_hist.set_title(f"{ticker} • Returns • Distribution")
    ax_hist.set_xlabel("Daily log return")
    ax_hist.set_ylabel("Density")

    r_clean = _numeric_clean(r)
    mean   = float(r_clean.mean()) if len(r_clean) else np.nan
    median = float(r_clean.median()) if len(r_clean) else np.nan
    mode_series = r_clean.mode()
    mode_val = float(mode_series.iloc[0]) if not mode_series.empty else np.nan
    std    = float(r_clean.std(ddof=1)) if len(r_clean) else np.nan
    skew   = float(stats.skew(r_clean, bias=False, nan_policy="omit")) if len(r_clean) else np.nan
    kurt   = float(stats.kurtosis(r_clean, fisher=True, bias=False, nan_policy="omit")) if len(r_clean) else np.nan

    stats_text = "\n".join([
        f"Mean:     {mean:.6f}"     if not np.isnan(mean) else "Mean:     n/a",
        f"Median:   {median:.6f}"   if not np.isnan(median) else "Median:   n/a",
        f"Mode:     {mode_val:.6f}" if not np.isnan(mode_val) else "Mode:     n/a",
        f"Std Dev:  {std:.6f}"      if not np.isnan(std) else "Std Dev:  n/a",
        f"Skewness: {skew:.4f}"     if not np.isnan(skew) else "Skewness: n/a",
        f"Kurtosis: {kurt:.4f}"     if not np.isnan(kurt) else "Kurtosis: n/a",
    ])
    ax_hist.text(
        0.02, 0.98, stats_text,
        transform=ax_hist.transAxes,
        fontsize=10,
        va="top", ha="left",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.85, edgecolor="gray")
    )

    # Guide lines: mean and ±1σ
    if not np.isnan(mean):
        ax_hist.axvline(mean, color="black", lw=1.4, linestyle="--", label="Mean")
    if not (np.isnan(mean) or np.isnan(std)):
        ax_hist.axvline(mean - std, color="gray", lw=1.0, linestyle="--", label="±1σ")
        ax_hist.axvline(mean + std, color="gray", lw=1.0, linestyle="--")
        handles, labels = ax_hist.get_legend_handles_labels()
        if labels:
            ax_hist.legend(loc="best", frameon=True)

    figs.append(fig_hist)

    # Q–Q plot
    fig_qq, ax_qq = plt.subplots()
    stats.probplot(r_clean, dist="norm", plot=ax_qq)
    ax_qq.set_title(f"{ticker} • Returns • Q–Q Plot vs Normal")
    figs.append(fig_qq)

    return figs

def _plot_returns_acf(r: pd.Series, ticker: str, max_lags: int = 20) -> plt.Figure:
    title = f"{ticker} • ACF • Returns"
    try:
        from statsmodels.graphics.tsaplots import plot_acf
        fig = plot_acf(r.dropna(), lags=max_lags)
        fig.axes[0].set_title(title)
        return fig
    except Exception:
        fig, ax = plt.subplots()
        lags = list(range(1, max_lags + 1))
        values = [r.corr(r.shift(k)) for k in lags]
        ax.bar(lags, values, color="steelblue")
        ax.axhline(0, color="gray", lw=0.8)
        ax.set_title(f"{title} (manual)")
        ax.set_xlabel("Lag")
        ax.set_ylabel("Autocorrelation")
        return fig

def _plot_rolling_autocorr(r: pd.Series, ticker: str, lag: int = 1, window: int = 20) -> plt.Figure:
    df_tmp = pd.DataFrame({"r": r}).dropna()
    df_tmp[f"r_lag_{lag}"] = df_tmp["r"].shift(lag)
    roll = df_tmp["r"].rolling(window).corr(df_tmp[f"r_lag_{lag}"])
    fig, ax = plt.subplots()
    ax.plot(roll.index, roll.values, color="forestgreen", lw=1.2)
    ax.axhline(0, color="gray", lw=0.8)
    ax.set_title(f"{ticker} • Rolling Autocorrelation (lag={lag}, window={window})")
    ax.set_xlabel("Date")
    ax.set_ylabel("Autocorr")
    return fig

def _plot_vol_clustering(r: pd.Series, ticker: str, max_lags: int = 20) -> List[plt.Figure]:
    """ACF of |r| and r^2 to reveal volatility clustering."""
    figs: List[plt.Figure] = []
    abs_r = r.abs()
    sq_r  = r.pow(2)

    # ACF |r|
    try:
        from statsmodels.graphics.tsaplots import plot_acf
        fig1 = plot_acf(abs_r.dropna(), lags=max_lags)
        fig1.axes[0].set_title(f"{ticker} • ACF • |Returns| (Volatility Clustering)")
        figs.append(fig1)
    except Exception:
        lags = list(range(1, max_lags + 1))
        values = [abs_r.corr(abs_r.shift(k)) for k in lags]
        fig1, ax1 = plt.subplots()
        ax1.bar(lags, values, color="indianred")
        ax1.axhline(0, color="gray", lw=0.8)
        ax1.set_title(f"{ticker} • ACF • |Returns| (manual)")
        figs.append(fig1)

    # ACF r^2
    try:
        from statsmodels.graphics.tsaplots import plot_acf
        fig2 = plot_acf(sq_r.dropna(), lags=max_lags)
        fig2.axes[0].set_title(f"{ticker} • ACF • Returns^2 (Volatility Clustering)")
        figs.append(fig2)
    except Exception:
        lags = list(range(1, max_lags + 1))
        values = [sq_r.corr(sq_r.shift(k)) for k in lags]
        fig2, ax2 = plt.subplots()
        ax2.bar(lags, values, color="firebrick")
        ax2.axhline(0, color="gray", lw=0.8)
        ax2.set_title(f"{ticker} • ACF • Returns^2 (manual)")
        figs.append(fig2)

    return figs

# ----------------------------
# Seasonality (derived from Date index)
# ----------------------------
def _seasonal_df_from_index(index_like: pd.Index) -> Optional[pd.DataFrame]:
    """Build seasonal fields from a Date-like index. Returns None if cannot coerce."""
    dt = pd.to_datetime(index_like, errors="coerce")
    if dt.isna().all():
        return None
    dt = pd.DatetimeIndex(dt)
    out = pd.DataFrame(
        {
            "month": dt.month,
            "day_of_week": dt.dayofweek,  # 0=Mon .. 6=Sun
            "quarter": dt.quarter,
            "year": dt.year,
            "is_month_end": dt.is_month_end.astype(int),
            "is_month_start": dt.is_month_start.astype(int),
        },
        index=index_like,
    )
    return out

def _plot_monthly_box_from_index(r: pd.Series, ticker: str) -> Optional[plt.Figure]:
    season = _seasonal_df_from_index(r.index)
    if season is None:
        return None
    d = pd.concat([r.rename("ret"), season["month"]], axis=1).dropna(subset=["ret"])
    d["month"] = pd.to_numeric(d["month"], errors="coerce").astype("Int64")
    d = d.dropna(subset=["month"])
    if d.empty:
        return None

    order = list(range(1, 13))
    fig, ax = plt.subplots()
    sns.boxplot(data=d, x="month", y="ret", ax=ax, order=order)
    ax.set_title(f"{ticker} • Monthly Returns")
    ax.set_xlabel("Month (1–12)")
    ax.set_ylabel("Daily log return")

    # Overlay mean by month + annotations
    grp = d.groupby("month")["ret"].mean()
    x_positions = ax.get_xticks()
    means_in_order = [float(grp.get(m, np.nan)) for m in order]

    ax.plot(x_positions, means_in_order, color="crimson", marker="D", lw=1.6,
            label="Average (mean)", zorder=5)

    for x, y in zip(x_positions, means_in_order):
        if not (np.isnan(y) or np.isinf(y)):
            ax.annotate(f"{y:+.4f}", (x, y),
                        textcoords="offset points", xytext=(0, 8),
                        ha="center", fontsize=9, color="crimson", fontweight="bold")

    handles, labels = ax.get_legend_handles_labels()
    if "Average (mean)" in labels:
        ax.legend(loc="best", frameon=True)

    return fig

def _plot_dow_box_from_index(r: pd.Series, ticker: str) -> Optional[plt.Figure]:
    season = _seasonal_df_from_index(r.index)
    if season is None:
        return None
    d = pd.concat([r.rename("ret"), season["day_of_week"]], axis=1).dropna(subset=["ret"])
    d["day_of_week"] = pd.to_numeric(d["day_of_week"], errors="coerce").astype("Int64")
    d = d.dropna(subset=["day_of_week"])
    if d.empty:
        return None

    labels = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
    order = list(range(0, 7))
    fig, ax = plt.subplots()
    sns.boxplot(data=d, x="day_of_week", y="ret", ax=ax, order=order)
    ax.set_title(f"{ticker} • Day-of-Week Returns")
    ax.set_xlabel("Day of Week")
    ax.set_xticklabels(labels, rotation=0)
    ax.set_ylabel("Daily log return")

    # Overlay mean by DOW + annotations
    grp = d.groupby("day_of_week")["ret"].mean()
    x_positions = ax.get_xticks()
    means_in_order = [float(grp.get(i, np.nan)) for i in order]

    ax.plot(x_positions, means_in_order, color="crimson", marker="D", lw=1.6,
            label="Average (mean)", zorder=5)

    for x, y in zip(x_positions, means_in_order):
        if not (np.isnan(y) or np.isinf(y)):
            ax.annotate(f"{y:+.4f}", (x, y),
                        textcoords="offset points", xytext=(0, 8),
                        ha="center", fontsize=9, color="crimson", fontweight="bold")

    handles, labels_ = ax.get_legend_handles_labels()
    if "Average (mean)" in labels_:
        ax.legend(loc="best", frameon=True)

    return fig

def _plot_quarter_box_from_index(r: pd.Series, ticker: str) -> Optional[plt.Figure]:
    season = _seasonal_df_from_index(r.index)
    if season is None:
        return None
    d = pd.concat([r.rename("ret"), season["quarter"]], axis=1).dropna(subset=["ret"])
    d["quarter"] = pd.to_numeric(d["quarter"], errors="coerce").astype("Int64")
    d = d.dropna(subset=["quarter"])
    if d.empty:
        return None
    fig, ax = plt.subplots()
    sns.boxplot(data=d, x="quarter", y="ret", ax=ax, order=[1, 2, 3, 4])
    ax.set_title(f"{ticker} • Quarterly Returns")
    ax.set_xlabel("Quarter")
    ax.set_ylabel("Daily log return")
    return fig

def _plot_month_year_heatmap_from_index(r: pd.Series, ticker: str) -> Optional[plt.Figure]:
    season = _seasonal_df_from_index(r.index)
    if season is None:
        return None
    d = pd.concat([r.rename("ret"), season[["month", "year"]]], axis=1).dropna(subset=["ret"])
    d["month"] = pd.to_numeric(d["month"], errors="coerce").astype("Int64")
    d["year"]  = pd.to_numeric(d["year"], errors="coerce").astype("Int64")
    d = d.dropna(subset=["month", "year"])
    if d.empty:
        return None
    piv = d.pivot_table(index="year", columns="month", values="ret", aggfunc="mean").sort_index()
    piv = piv.reindex(columns=list(range(1, 13)))
    fig, ax = plt.subplots()
    sns.heatmap(piv, cmap="RdYlGn", center=0, annot=False, ax=ax)
    ax.set_title(f"{ticker} • Month×Year Heatmap (Avg Daily Returns)")
    ax.set_xlabel("Month")
    ax.set_ylabel("Year")
    return fig

def _plot_month_end_start_bars_from_index(r: pd.Series, ticker: str) -> Optional[plt.Figure]:
    season = _seasonal_df_from_index(r.index)
    if season is None:
        return None
    d = pd.concat([r.rename("ret"), season[["is_month_end", "is_month_start"]]], axis=1).dropna(subset=["ret"])
    d["is_month_end"]   = pd.to_numeric(d["is_month_end"], errors="coerce").astype("Int64")
    d["is_month_start"] = pd.to_numeric(d["is_month_start"], errors="coerce").astype("Int64")
    d = d.dropna(subset=["is_month_end", "is_month_start"])
    if d.empty:
        return None

    rows = []
    for col, label in [("is_month_end", "Month End"), ("is_month_start", "Month Start")]:
        grp = d.groupby(col)["ret"].mean()
        rows.append({"Group": f"{label}:Yes", "AvgRet": float(grp.get(1, np.nan))})
        rows.append({"Group": f"{label}:No",  "AvgRet": float(grp.get(0, np.nan))})
    res = pd.DataFrame(rows)

    fig, ax = plt.subplots()
    sns.barplot(data=res, x="Group", y="AvgRet", palette="Blues_d", ax=ax)
    ax.axhline(0, color="gray", lw=0.8)
    ax.set_title(f"{ticker} • Avg Returns: Month-End/Start vs Others")
    ax.set_xlabel("")
    ax.set_ylabel("Average Daily Log Return")
    ax.tick_params(axis="x", rotation=15)
    return fig

# ----------------------------
# Engineered Feature Visuals
# ----------------------------
def _plot_feature_distributions(df: pd.DataFrame, cols: List[str], title_prefix: str, ticker: str) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    cols_num = [c for c in cols if c in df.columns and _numeric_clean(df[c]).size > 0]
    if not cols_num:
        return figs
    chunk_size = 6
    for i in range(0, len(cols_num), chunk_size):
        batch = cols_num[i:i + chunk_size]
        rows = int(np.ceil(len(batch) / 3))
        fig, axes = plt.subplots(rows, 3, figsize=(11, max(3.5, 3.2 * rows)))
        axes = np.array(axes).reshape(-1)
        for j, c in enumerate(batch):
            s = _numeric_clean(df[c])
            sns.histplot(s, bins=50, kde=True, ax=axes[j], color="teal")
            axes[j].set_title(c)
        for k in range(len(batch), len(axes)):
            fig.delaxes(axes[k])
        fig.suptitle(f"{ticker} • {title_prefix}", x=0.06, y=0.99, ha="left", fontsize=14)
        figs.append(fig)
    return figs

def _plot_feature_corr(df: pd.DataFrame, cols: List[str], title: str, ticker: str) -> Optional[plt.Figure]:
    cols_num = [c for c in cols if c in df.columns and _numeric_clean(df[c]).size > 0]
    if len(cols_num) < 2:
        return None
    data = df[cols_num].apply(pd.to_numeric, errors="coerce")
    corr = data.corr()
    fig, ax = plt.subplots()
    sns.heatmap(corr, cmap="viridis", center=0, annot=True, fmt=".2f", ax=ax)
    ax.set_title(f"{ticker} • {title}", loc="left")
    return fig

# ----------------------------
# Per-ticker report
# ----------------------------
def eda_single_ticker(name: str, df: pd.DataFrame) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    ticker = _safe_title(name)
    df = df.copy().sort_index()

    # Price series
    price_col = _adj_or_close_col(df, ticker)
    if price_col:
        price = _numeric_clean(df[price_col])
        if not price.empty:
            figs.append(_plot_price_series(price, ticker))
            figs.append(_plot_price_acf(price, ticker, max_lags=20))
            figs.append(_plot_mas(price, ticker))
    else:
        info_df = pd.DataFrame({"Message": [f"No Adj/Close column found for {ticker}."]})
        figs.append(_table_figure_from_dataframe(info_df, f"{ticker} • Price Not Available"))

    # Returns-driven stylized facts
    r = _get_return_series(df, ticker)
    if r is not None and not r.empty:
        figs.append(_plot_returns_ts(r, ticker))
        figs.extend(_plot_returns_hist_qq(r, ticker))
        figs.append(_plot_returns_acf(r, ticker, max_lags=20))
        figs.append(_plot_rolling_autocorr(r, ticker, lag=1, window=20))
        figs.extend(_plot_vol_clustering(r, ticker, max_lags=20))

        # Seasonality — derived from the Date index of r
        fig_m = _plot_monthly_box_from_index(r, ticker);          figs.append(fig_m) if fig_m else None
        fig_d = _plot_dow_box_from_index(r, ticker);              figs.append(fig_d) if fig_d else None
        fig_q = _plot_quarter_box_from_index(r, ticker);          figs.append(fig_q) if fig_q else None
        fig_h = _plot_month_year_heatmap_from_index(r, ticker);   figs.append(fig_h) if fig_h else None
        fig_b = _plot_month_end_start_bars_from_index(r, ticker); figs.append(fig_b) if fig_b else None
    else:
        info_df = pd.DataFrame({"Message": [f"No return series found for {ticker}."]})
        figs.append(_table_figure_from_dataframe(info_df, f"{ticker} • Returns Not Available"))

    # Engineered feature plots
    feat = _feature_cols_for_ticker(df, ticker)
    figs.extend(_plot_feature_distributions(df, feat["spreads"], "Spreads", ticker))
    figs.extend(_plot_feature_distributions(df, feat["ma"], "Moving Averages / EMAs", ticker))
    figs.extend(_plot_feature_distributions(df, feat["lags"], "Lagged Prices", ticker))

    fm = _plot_feature_corr(df, feat["ma"], "Correlation • Moving Averages", ticker)
    if fm:
        figs.append(fm)
    fm2 = _plot_feature_corr(df, feat["spreads"] + feat["lags"], "Correlation • Spreads + Lags", ticker)
    if fm2:
        figs.append(fm2)

    return figs

# ----------------------------
# Cross-ticker overlay (Adj/Close)
# ----------------------------
def eda_cross_ticker_adjclose(frames: Dict[str, pd.DataFrame]) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    price_cols: Dict[str, pd.Series] = {}

    for name, df in frames.items():
        t = _safe_title(name)
        price_col = _adj_or_close_col(df, t)
        if price_col:
            series = _numeric_clean(df[price_col])
            if not series.empty:
                price_cols[t] = series

    if price_cols:
        prices = pd.DataFrame(price_cols).sort_index()
        fig, ax = plt.subplots()
        for name in prices.columns:
            ax.plot(prices.index, prices[name], lw=1.5, label=name)
        ax.set_title("Cross-Ticker • Adjusted/Close Prices (Raw Levels)")
        ax.set_xlabel("Date")
        ax.set_ylabel("Price / Level")
        ax.legend(loc="best")
        figs.append(fig)

    return figs

# ----------------------------
# Report Orchestration
# ----------------------------
def build_multi_asset_report(
    frames: Dict[str, pd.DataFrame],
    pdf_title: str = "Multi-Asset EDA report.pdf",
    show_plots: bool = True
) -> None:
    """
    Build a multi-page PDF report covering stylized facts & seasonality
    for single-ticker frames.

    Seasonality is derived from the Date index (DatetimeIndex).
    """
    ordered_items = sorted(frames.items(), key=lambda kv: kv[0])
    figs: List[plt.Figure] = []

    # Per-ticker
    for name, df in ordered_items:
        figs.extend(eda_single_ticker(name, df))

    # Cross-ticker overlay
    figs.extend(eda_cross_ticker_adjclose(frames))

    with PdfPages(pdf_title) as pdf:
        # Cover page
        cover_fig, ax = plt.subplots()
        ax.axis("off")
        ax.text(0.0, 0.9, "Multi-Asset EDA Report", fontsize=22, fontweight="bold",
                ha="left", va="top")
        tickers_list = ", ".join([_safe_title(k) for k, _ in ordered_items])
        ax.text(0.0, 0.78, f"Tickers: {tickers_list}", fontsize=12, ha="left", va="top")
        ax.text(0.0, 0.72, "Contents:", fontsize=14, fontweight="bold", ha="left", va="top")
        contents = [
            "• Price series & ACF(Price), Moving Averages (5/10/20)",
            "• Daily log-returns time series; Histogram+KDE (with Std Dev); Q–Q plot",
            "• ACF(Returns), Rolling Autocorr(Returns)",
            "• Volatility clustering: ACF(|r|) & ACF(r^2)",
            "• Seasonality (from Date index): Monthly/DOW/Quarterly boxplots; Month×Year heatmap; Month-End/Start bars",
            "• Feature distributions (Spreads/MAs/Lags) & correlation heatmaps",
            "• Cross-ticker adjusted/close overlay",
        ]
        for i, line in enumerate(contents):
            ax.text(0.02, 0.66 - i * 0.05, line, fontsize=12, ha="left", va="top")
        pdf.savefig(cover_fig)
        plt.close(cover_fig)

        # Content pages
        for fig in figs:
            pdf.savefig(fig)
            plt.close(fig)

    print(f"✅ PDF saved: {pdf_title}")

    if show_plots:
        # Recreate for display (since figures were closed after saving)
        display_figs: List[plt.Figure] = []
        for name, df in ordered_items:
            display_figs.extend(eda_single_ticker(name, df))
        display_figs.extend(eda_cross_ticker_adjclose(frames))
        for fig in display_figs:
            plt.show()

# ----------------------------
# Backward-compatible alias (optional)
# ----------------------------
def build_energy_etf_report(frames: Dict[str, pd.DataFrame],
                            pdf_title: str = "EDA energy ETF report.pdf",
                            show_plots: bool = True) -> None:
    """Alias to keep older notebooks working."""
    build_multi_asset_report(frames, pdf_title=pdf_title, show_plots=show_plots)

# ----------------------------
# Example usage (your requested frames)
# ----------------------------
if __name__ == "__main__":
    # DataFrames are assumed to already exist from earlier cells/code:
    # spy_df, gld_df, clf_df, psx_df, vlo_df, mpc_df, eurusd_df

    frames = {
        "SPY": spy_df,
        "GLD": gld_df,
        "CL=F": clf_df,
        "PSX": psx_df,
        "VLO": vlo_df,
        "MPC": mpc_df,
        "EUR_USD": eurusd_df,
    }

    build_multi_asset_report(frames, pdf_title="Multi-Asset Stylized Facts report.pdf", show_plots=False)

/tmp/ipykernel_11276/866163125.py:401: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(labels, rotation=0)
/tmp/ipykernel_11276/866163125.py:478: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=res, x="Group", y="AvgRet", palette="Blues_d", ax=ax)
/tmp/ipykernel_11276/866163125.py:194: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
/tmp/ipykernel_11276/866163125.py:401: UserWarning: set_ticklabels() should only be used with a fixed number of ticks,

✅ PDF saved: Multi-Asset Stylized Facts report.pdf


## Generate GARCH Report

In [7]:
# -*- coding: utf-8 -*-
"""
Univariate GARCH(1,1) Volatility & Diagnostics Report (Multi-Asset)

Updates included:
  • h-step forecast horizon default set to 90 days.
  • Volatility series are shown as ANNUALIZED PERCENT values so they can be compared.
  • Adds 100-day realized volatility trend (annualized %) overlaid on conditional volatility plot.
  • Removes lag-0 (day 0) from all ACF plots.
  • ACF plots include an explicit explanation of the shaded region (significance band).
  • ACF plots auto-scale the y-axis based on data (and CI band) for readability.
  • Metrics table now includes a simple explanation for each metric.

Inputs:
    Assumes we already have engineered feature DataFrames in memory, keyed by asset name,
    e.g.: spy_df, gld_df, clf_df, psx_df, vlo_df, mpc_df, eurusd_df

Output (default):
    "Univariate_GARCH_Volatility_Report.pdf"
"""

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import textwrap  # <-- ADDED (for table cell wrapping)

from typing import Dict, Optional, List, Tuple
from matplotlib.backends.backend_pdf import PdfPages
from scipy import stats

# --- econometrics ---
from arch import arch_model
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf as sm_acf

# ----------------------------
# Global settings
# ----------------------------
TRADING_DAYS_PER_YEAR = 252
REALIZED_VOL_WINDOW_DAYS = 100
DEFAULT_FORECAST_H = 90

# ACF plotting defaults
ACF_ALPHA = 0.05  # 95% significance band
ACF_LAGS_DEFAULT = 20

# ----------------------------
# Plot styling
# ----------------------------
plt.rcParams["figure.figsize"] = (11, 7)
plt.rcParams["axes.grid"] = True
plt.rcParams["figure.autolayout"] = True
sns.set_style("whitegrid")

# ----------------------------
# Helpers
# ----------------------------
def _safe_title(name: str) -> str:
    return str(name).strip() if name else "Unknown"


def _numeric_clean(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce").astype(float)


def _infer_engineered_logret_col(df: pd.DataFrame, asset_name: str) -> Optional[str]:
    """
    Prefer exact engineered naming: log_ret|{asset_name}
    Fallback: any column that starts with 'log_ret|' (first match).
    """
    exact = f"log_ret|{asset_name}"
    if exact in df.columns:
        return exact
    for c in df.columns:
        if str(c).lower().startswith("log_ret|"):
            return c
    return None


def _infer_price_col(df: pd.DataFrame, asset_name: str) -> Optional[str]:
    """
    Prefer: Adj Close|{asset}, else Close|{asset}
    Fallback: any column containing 'adj close' else any containing 'close'
    """
    direct = [f"Adj Close|{asset_name}", f"Close|{asset_name}"]
    for c in direct:
        if c in df.columns:
            return c

    for c in df.columns:
        if "adj close" in str(c).lower():
            return c
    for c in df.columns:
        if "close" in str(c).lower():
            return c
    return None


def _get_returns(df: pd.DataFrame, asset_name: str) -> pd.Series:
    """
    Get engineered log returns if present; else compute from inferred Adj/Close.
    Returns a pandas Series with a DatetimeIndex if possible.
    """
    col = _infer_engineered_logret_col(df, asset_name)
    if col is not None:
        r = _numeric_clean(df[col]).dropna().sort_index()
        if r.size > 0:
            return r

    px_col = _infer_price_col(df, asset_name)
    if px_col is None:
        return pd.Series(dtype=float)

    p = _numeric_clean(df[px_col]).sort_index()
    p = p.replace([0, np.inf, -np.inf], np.nan)
    return np.log(p / p.shift(1)).dropna()


def _qqplot_fig(series: pd.Series, title: str) -> plt.Figure:
    fig, ax = plt.subplots()
    stats.probplot(series.dropna(), dist="norm", plot=ax)
    ax.set_title(title)
    return fig


def _table_figure_from_df(title: str, df: pd.DataFrame) -> plt.Figure:
    """
    Render a DataFrame as a nicely formatted matplotlib table.

    UPDATED:
      - Wraps text within cells so long descriptions don't overflow.
      - Adjusts column widths and row heights to accommodate wrapped text.
    """
    # dynamic height
    fig_h = min(9, 1.2 + 0.38 * (len(df) + 2))
    fig, ax = plt.subplots(figsize=(11, fig_h))
    ax.axis("off")
    ax.set_title(title, fontsize=14, loc="left")

    # ---- Wrap text content per-column (character-based) ----
    # You can tweak these widths if you want different wrapping behavior.
    wrap_width_by_col = {
        0: 24,   # Metric
        1: 18,   # Value
        2: 70,   # Explanation (long)
    }

    def _wrap_cell(val: object, col_idx: int) -> str:
        s = "" if val is None else str(val)
        s = s.replace("\r\n", "\n").replace("\r", "\n")
        width = wrap_width_by_col.get(col_idx, 30)
        # Preserve explicit newlines while wrapping each line.
        lines = []
        for line in s.split("\n"):
            if line.strip() == "":
                lines.append("")
            else:
                lines.append(textwrap.fill(line, width=width, break_long_words=False))
        return "\n".join(lines)

    cell_text = []
    for row in df.values.tolist():
        wrapped_row = []
        for j, v in enumerate(row):
            wrapped_row.append(_wrap_cell(v, j))
        cell_text.append(wrapped_row)

    col_labels = [str(c) for c in df.columns]

    tbl = ax.table(
        cellText=cell_text,
        colLabels=col_labels,
        loc="center",
        cellLoc="left",
    )

    # Font and scale
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(9.5)
    tbl.scale(1, 1.18)

    # ---- Set column widths (fractions of axes width) ----
    # If there are only 2 columns (Metric/Value), keep it balanced.
    ncols = len(df.columns)
    if ncols == 3:
        col_widths = {0: 0.23, 1: 0.17, 2: 0.60}
    elif ncols == 2:
        col_widths = {0: 0.38, 1: 0.62}
    else:
        # fallback: equal widths
        col_widths = {j: 1.0 / max(ncols, 1) for j in range(ncols)}

    # Apply widths and enable text wrapping in every cell
    for (r, c), cell in tbl.get_celld().items():
        if c in col_widths:
            cell.set_width(col_widths[c])
        cell.get_text().set_wrap(True)

        # Header styling (row 0 in matplotlib table is header)
        if r == 0:
            cell.set_facecolor("#F2F2F2")
            cell.get_text().set_weight("bold")

    # ---- Adjust row heights based on wrapped line count ----
    # Determine max number of lines per row (including header)
    base_h = tbl[(1, 0)].get_height() if (1, 0) in tbl.get_celld() else 0.05
    for r in range(1, len(df) + 1):  # data rows start at 1
        max_lines = 1
        for c in range(ncols):
            txt = tbl[(r, c)].get_text().get_text()
            max_lines = max(max_lines, txt.count("\n") + 1)
        # Increase height proportionally (tuned factor)
        new_h = base_h * (0.95 + 0.55 * (max_lines - 1))
        for c in range(ncols):
            tbl[(r, c)].set_height(new_h)

    return fig


def _table_figure_from_dict(title: str, rows: Dict[str, object]) -> plt.Figure:
    df = pd.DataFrame({"Metric": list(rows.keys()),
                       "Value": [rows[k] for k in rows]})
    return _table_figure_from_df(title, df)


def _half_life(alpha: float, beta: float) -> Optional[float]:
    """Half-life in days: ln(0.5) / ln(alpha+beta), if 0 < alpha+beta < 1."""
    rho = alpha + beta
    if rho <= 0 or rho >= 1:
        return None
    return math.log(0.5) / math.log(rho)


def _annualize_vol_to_pct(daily_vol: pd.Series, scale: int = TRADING_DAYS_PER_YEAR) -> pd.Series:
    """Convert daily volatility (std dev of daily returns) to annualized %."""
    return daily_vol * np.sqrt(scale) * 100.0


def _annualize_var_to_pct2(daily_var: pd.Series, scale: int = TRADING_DAYS_PER_YEAR) -> pd.Series:
    """Convert daily variance to annualized percent^2 (variance units)."""
    return daily_var * scale * (100.0 ** 2)


def _format_number(x, ndigits: int = 6) -> str:
    if x is None:
        return "—"
    try:
        if isinstance(x, (float, np.floating, int, np.integer)) and not np.isnan(float(x)):
            # use scientific for very small or very large
            ax = abs(float(x))
            if ax != 0 and (ax < 1e-4 or ax > 1e4):
                return f"{float(x):.{ndigits}e}"
            return f"{float(x):.{ndigits}f}"
    except Exception:
        pass
    return str(x)


def _metrics_table_with_explanations(title: str, metrics: Dict[str, object], dist: str) -> plt.Figure:
    """
    Build a metrics table with a plain-language explanation column.
    """
    explanations = {
        "μ (mean)": "Estimated average daily return (constant mean in the return equation).",
        "ω": "Variance intercept; baseline level feeding the long-run variance.",
        "α": "Shock (ARCH) effect; how strongly yesterday’s squared residual increases today’s variance.",
        "β": "Persistence (GARCH) effect; how strongly yesterday’s variance carries into today.",
        "ν (Student‑t df)": "Student‑t degrees of freedom (lower ν = heavier tails). Only relevant for Student‑t innovations.",
        "α+β (persistence)": "Total variance persistence; closer to 1 implies slower mean reversion and stronger volatility clustering.",
        "σ∞² = ω/(1−α−β)": "Long-run (unconditional) variance implied by the model, assuming α+β < 1.",
        "Half‑life (days)": "Approx. days for a volatility shock to decay by 50% (based on α+β).",
        "log‑likelihood": "Model fit objective value under maximum likelihood; higher is better (within same data/model).",
        "AIC": "Akaike Information Criterion (penalized fit); lower is better for comparing models on the same data.",
        "BIC": "Bayesian Information Criterion (stronger penalty than AIC); lower is better for comparing models on the same data."
    }

    rows = []
    for k, v in metrics.items():
        # keep Student-t row meaningful
        if k == "ν (Student‑t df)" and dist != "t":
            expl = "Not estimated under Gaussian innovations."
        else:
            expl = explanations.get(k, "")
        rows.append([k, _format_number(v, ndigits=6), expl])

    df = pd.DataFrame(rows, columns=["Metric", "Value", "Explanation"])
    return _table_figure_from_df(title, df)

# ----------------------------
# ACF plotting with shaded-band explanation + autoscaling + lag0 omitted
# ----------------------------
def _annotate_acf_band(ax: plt.Axes, alpha: float) -> None:
    """
    Add an on-plot explanation for the shaded confidence band.
    """
    pct = int(round((1.0 - alpha) * 100))
    note = (
        f"Shaded band: ±{pct}% significance bounds around 0 under H₀ (no autocorrelation).\n"
        f"Bars outside the band are statistically significant at α={alpha:.2f}."
    )
    ax.text(
        0.02, 0.02, note,
        transform=ax.transAxes,
        va="bottom", ha="left",
        fontsize=9,
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.85, edgecolor="lightgray")
    )


def _set_acf_ylim(ax: plt.Axes, y_values: np.ndarray, ci_low: Optional[np.ndarray] = None, ci_high: Optional[np.ndarray] = None) -> None:
    """
    Auto-scale y-axis based on data and (optional) CI band, with some padding.
    """
    arrs = [np.asarray(y_values)]
    if ci_low is not None:
        arrs.append(np.asarray(ci_low))
    if ci_high is not None:
        arrs.append(np.asarray(ci_high))
    mx = float(np.nanmax(np.abs(np.concatenate([a[~np.isnan(a)] for a in arrs if a.size > 0])))) if arrs else 1.0
    mx = max(mx, 0.05)  # prevent too-tight scale when correlations are tiny
    pad = 1.25 * mx
    ax.set_ylim(-pad, pad)


def _plot_acf_omit_lag0(x: pd.Series, lags: int, title: str, alpha: float = ACF_ALPHA) -> plt.Figure:
    """
    Plot ACF excluding lag 0 (day 0), with:
      - shaded confidence band explanation
      - auto y-axis scaling
      - consistent behavior across statsmodels versions

    Strategy:
      1) Try statsmodels.graphics.tsaplots.plot_acf with zero=False, alpha=alpha, auto_ylims=True.
      2) If options are unsupported (older statsmodels), manually compute acf + confint using
         statsmodels.tsa.stattools.acf and plot with band centered at 0.
    """
    x_clean = pd.Series(x).dropna().astype(float)

    # --- attempt using plot_acf (best-looking, includes band) ---
    try:
        fig = plot_acf(
            x_clean,
            lags=lags,
            zero=False,           # omit lag 0
            alpha=alpha,          # show band
            auto_ylims=True       # auto scale y based on values & band (if supported)
        )
        ax = fig.axes[0]
        ax.set_title(title)
        ax.set_xlabel("Lag")
        ax.set_ylabel("Autocorrelation")
        _annotate_acf_band(ax, alpha=alpha)
        return fig
    except TypeError:
        # --- manual fallback: compute acf + confint and plot with shaded band ---
        acf_vals, confint = sm_acf(x_clean.values, nlags=lags, alpha=alpha, fft=True)

        # omit lag 0
        lag_idx = np.arange(1, lags + 1)
        vals = acf_vals[1:lags + 1]
        ci = confint[1:lags + 1, :]  # [low, high] for each lag

        # statsmodels plot_acf shades confint centered at 0: (confint - acf_vals)
        band_low = ci[:, 0] - vals
        band_high = ci[:, 1] - vals

        fig, ax = plt.subplots(figsize=(11, 6))
        ax.axhline(0.0, color="gray", lw=0.8)

        # shade CI band
        ax.fill_between(
            lag_idx.astype(float),
            band_low,
            band_high,
            color="C0",
            alpha=0.20,
            edgecolor="none"
        )

        # stem-like plot
        ax.vlines(lag_idx, 0.0, vals, colors="C0", lw=1.2)
        ax.plot(lag_idx, vals, "o", color="C0", markersize=4)

        ax.set_title(title)
        ax.set_xlabel("Lag")
        ax.set_ylabel("Autocorrelation")

        _set_acf_ylim(ax, vals, band_low, band_high)
        ax.set_xlim(0.5, lags + 0.5)
        _annotate_acf_band(ax, alpha=alpha)

        return fig

# ----------------------------
# GARCH(1,1) estimation per asset
# ----------------------------
def fit_garch11(
    returns: pd.Series,
    dist: str = "normal",          # "normal" or "t"
    robust_se: bool = True
) -> Tuple[object, Dict[str, object], pd.Index]:
    """
    Estimate GARCH(1,1) with mean=constant (μ), variance=ω+α e_{t-1}^2 + β σ_{t-1}^2.
    Returns fitted result, metrics dict, and the fit index for plotting.
    """
    r = returns.dropna().astype(float)
    if r.empty:
        raise ValueError("Empty returns series provided to fit_garch11().")

    fit_index = r.index

    am = arch_model(
        r,
        mean="constant",
        vol="GARCH",
        p=1, q=1,
        dist="normal" if dist == "normal" else "t",
        rescale=False,
    )

    fit_kwargs = dict(disp="off", update_freq=0, show_warning=False)
    if robust_se:
        fit_kwargs["cov_type"] = "robust"

    res = am.fit(**fit_kwargs)

    params = res.params
    mu = float(params.get("mu", np.nan))
    omega = float(params.get("omega", np.nan))
    alpha = float(params.get("alpha[1]", np.nan))
    beta  = float(params.get("beta[1]", np.nan))
    nu    = float(params.get("nu", np.nan)) if dist == "t" and "nu" in params.index else np.nan

    rho = alpha + beta
    sig2_inf = np.nan
    if (1 - rho) > 1e-12:
        sig2_inf = omega / (1 - rho)

    hl = _half_life(alpha, beta)

    metrics = {
        "μ (mean)": mu,
        "ω": omega,
        "α": alpha,
        "β": beta,
        "ν (Student‑t df)": nu if dist == "t" else "—",
        "α+β (persistence)": rho,
        "σ∞² = ω/(1−α−β)": sig2_inf,
        "Half‑life (days)": hl,
        "log‑likelihood": float(res.loglikelihood),
        "AIC": float(res.aic),
        "BIC": float(res.bic),
    }
    return res, metrics, fit_index

# ----------------------------
# Plots
# ----------------------------
def _plot_volatility_path(
    res,
    fit_index: pd.Index,
    returns: pd.Series,
    title_prefix: str,
    realized_window: int = REALIZED_VOL_WINDOW_DAYS
) -> plt.Figure:
    """
    Plot conditional variance (annualized %^2) and volatility (annualized %),
    and overlay realized volatility (rolling window, annualized %).
    """
    sig_daily = pd.Series(np.asarray(res.conditional_volatility), index=fit_index).dropna()

    sig_ann_pct = _annualize_vol_to_pct(sig_daily)
    sig2_ann_pct2 = _annualize_var_to_pct2(sig_daily ** 2)

    r_aligned = pd.Series(returns).reindex(fit_index).astype(float)
    realized_ann_pct = _annualize_vol_to_pct(
        r_aligned.rolling(realized_window).std(ddof=1)
    )

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(11, 8), sharex=True)

    ax1.plot(sig2_ann_pct2.index, sig2_ann_pct2.values, color="firebrick", lw=1.2)
    ax1.set_title(f"{title_prefix} • Conditional Variance (Annualized, %²)")
    ax1.set_ylabel("Annualized Variance (%²)")

    ax2.plot(sig_ann_pct.index, sig_ann_pct.values, color="royalblue", lw=1.2,
             label="Conditional Vol (GARCH, ann. %)")
    ax2.plot(realized_ann_pct.index, realized_ann_pct.values, color="darkorange", lw=1.1, ls="--",
             label=f"Realized Vol ({realized_window}D, ann. %)")

    ax2.set_title(f"{title_prefix} • Volatility (Annualized %) • Conditional vs Realized")
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Annualized Volatility (%)")
    ax2.legend(loc="best")

    fig.tight_layout()
    return fig


# --- ADDED: plot daily percent returns together with conditional volatility ---
def _plot_daily_pct_returns_with_conditional_vol(
    res,
    fit_index: pd.Index,
    returns: pd.Series,
    title_prefix: str
) -> plt.Figure:
    """
    Plot DAILY percent returns together with CONDITIONAL volatility.

    - Left axis: daily returns in percent (100 * log return)
    - Right axis: conditional volatility annualized percent (daily sigma * sqrt(252) * 100)

    Uses a twin y-axis so both series are readable on their natural scales.
    """
    r_aligned = pd.Series(returns).reindex(fit_index).astype(float)
    r_pct = 100.0 * r_aligned

    sig_daily = pd.Series(np.asarray(res.conditional_volatility), index=fit_index).dropna()
    sig_ann_pct = _annualize_vol_to_pct(sig_daily)

    # Align on common index to avoid plotting gaps/misalignment
    common_idx = r_pct.index.intersection(sig_ann_pct.index)
    r_pct = r_pct.reindex(common_idx)
    sig_ann_pct = sig_ann_pct.reindex(common_idx)

    fig, ax1 = plt.subplots(figsize=(11, 7))

    ax1.plot(r_pct.index, r_pct.values, color="slategray", lw=0.9, alpha=0.75, label="Daily Return (%)")
    ax1.axhline(0.0, color="gray", lw=0.8)
    ax1.set_ylabel("Daily Return (%)", color="slategray")
    ax1.tick_params(axis="y", labelcolor="slategray")

    ax2 = ax1.twinx()
    ax2.plot(sig_ann_pct.index, sig_ann_pct.values, color="royalblue", lw=1.2, label="Conditional Vol (ann. %)")
    ax2.set_ylabel("Conditional Volatility (Annualized %)", color="royalblue")
    ax2.tick_params(axis="y", labelcolor="royalblue")

    ax1.set_title(f"{title_prefix} • Daily % Returns vs Conditional Volatility (Annualized %)")
    ax1.set_xlabel("Date")

    # Combined legend
    lines1, labels1 = ax1.get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 + lines2, labels1 + labels2, loc="best")

    fig.tight_layout()
    return fig
# --- END ADDED ---


def _plot_std_residuals(res, fit_index: pd.Index, title_prefix: str) -> plt.Figure:
    z = pd.Series(np.asarray(res.std_resid), index=fit_index).dropna()
    fig, ax = plt.subplots()
    ax.plot(z.index, z.values, lw=1.0, color="slategray")
    ax.axhline(0, color="gray", lw=0.8)
    ax.set_title(f"{title_prefix} • Standardized Residuals")
    ax.set_xlabel("Date")
    return fig


def _plot_acf_resid_sq(res, fit_index: pd.Index, lags: int, title_prefix: str) -> plt.Figure:
    z = pd.Series(np.asarray(res.std_resid), index=fit_index).dropna()
    z2 = z ** 2
    return _plot_acf_omit_lag0(
        z2,
        lags=lags,
        title=f"{title_prefix} • ACF(Standardized Residuals²) [Lag 0 Omitted]",
        alpha=ACF_ALPHA
    )


def _variance_forecast_path(res, fit_index: pd.Index, h: int, title_prefix: str) -> plt.Figure:
    """
    Closed-form h-step VARIANCE forecast for GARCH(1,1):
      σ_{T+k|T}^2 = σ∞² + (α+β)^k (σ_T^2 − σ∞²)

    Plotted as implied h-step VOLATILITY forecast (annualized %).
    """
    params = res.params
    omega = float(params.get("omega", np.nan))
    alpha = float(params.get("alpha[1]", np.nan))
    beta  = float(params.get("beta[1]", np.nan))
    rho   = alpha + beta

    sig_daily = pd.Series(np.asarray(res.conditional_volatility), index=fit_index).dropna()
    last_sig2 = float(sig_daily.iloc[-1] ** 2)

    if (1 - rho) <= 1e-12 or np.isnan(omega) or np.isnan(rho):
        f_var_path = np.full(h, last_sig2)
    else:
        sig2_inf = omega / (1 - rho)
        f_var_path = np.array([
            sig2_inf + (rho ** k) * (last_sig2 - sig2_inf) for k in range(1, h + 1)
        ])

    f_vol_ann_pct = np.sqrt(np.maximum(f_var_path, 0.0)) * np.sqrt(TRADING_DAYS_PER_YEAR) * 100.0

    fig, ax = plt.subplots()
    ax.plot(range(1, h + 1), f_vol_ann_pct, marker="o", lw=1.2, color="darkorange")
    ax.set_title(f"{title_prefix} • h‑Step Volatility Forecast (Annualized %, Closed‑Form)")
    ax.set_xlabel("h (days ahead)")
    ax.set_ylabel("Annualized Volatility (%)")
    return fig


def _stylized_facts_plots(r: pd.Series, title_prefix: str) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    r_clean = r.dropna().astype(float)

    fig1, ax1 = plt.subplots()
    sns.histplot(r_clean, bins=60, kde=True, stat="density", color="teal", ax=ax1)
    ax1.set_title(f"{title_prefix} • Return Distribution")
    ax1.set_xlabel("Daily log return")

    mean, std = float(r_clean.mean()), float(r_clean.std(ddof=1))
    skew = stats.skew(r_clean, bias=False)
    kurt = stats.kurtosis(r_clean, fisher=True, bias=False)

    box = "\n".join([
        f"Mean: {mean:.6f}",
        f"Std:  {std:.6f}",
        f"Skew: {skew:.3f}",
        f"Kurt: {kurt:.3f}",
    ])
    ax1.text(
        0.02, 0.98, box, transform=ax1.transAxes, va="top", ha="left",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.85)
    )
    ax1.axvline(mean, color="black", ls="--", lw=1.2, label="Mean")
    ax1.axvline(mean - std, color="gray", ls="--", lw=1.0)
    ax1.axvline(mean + std, color="gray", ls="--", lw=1.0)
    ax1.legend(loc="best")
    figs.append(fig1)

    figs.append(_qqplot_fig(r_clean, f"{title_prefix} • Q–Q Plot (Normal)"))

    figs.append(_plot_acf_omit_lag0(r_clean.abs(), lags=ACF_LAGS_DEFAULT,
                                   title=f"{title_prefix} • ACF(|r|) [Lag 0 Omitted]", alpha=ACF_ALPHA))
    figs.append(_plot_acf_omit_lag0((r_clean ** 2), lags=ACF_LAGS_DEFAULT,
                                   title=f"{title_prefix} • ACF(r²) [Lag 0 Omitted]", alpha=ACF_ALPHA))

    return figs

# ----------------------------
# Per-asset workflow
# ----------------------------
def garch_diagnostics_for_asset(
    asset_name: str,
    df: pd.DataFrame,
    dist: str = "normal",
    forecast_h: int = DEFAULT_FORECAST_H
) -> List[plt.Figure]:
    figs: List[plt.Figure] = []
    title_prefix = _safe_title(asset_name)

    r = _get_returns(df, asset_name)
    if r.empty:
        figs.append(_table_figure_from_dict(
            f"{title_prefix} • Data Warning",
            {"Message": "No returns found or computed for this asset."}
        ))
        return figs

    res, metrics, fit_index = fit_garch11(r, dist=dist, robust_se=True)

    # Table with metric explanations
    figs.append(_metrics_table_with_explanations(f"{title_prefix} • GARCH(1,1) Estimates", metrics, dist=dist))

    figs.extend(_stylized_facts_plots(r, title_prefix))

    # --- ADDED: daily percent returns plotted together with conditional volatility ---
    figs.append(_plot_daily_pct_returns_with_conditional_vol(res, fit_index, returns=r, title_prefix=title_prefix))

    figs.append(_plot_volatility_path(res, fit_index, returns=r, title_prefix=title_prefix,
                                      realized_window=REALIZED_VOL_WINDOW_DAYS))
    figs.append(_plot_std_residuals(res, fit_index, title_prefix))
    figs.append(_plot_acf_resid_sq(res, fit_index, ACF_LAGS_DEFAULT, title_prefix))
    figs.append(_variance_forecast_path(res, fit_index, forecast_h, title_prefix))

    return figs

# ----------------------------
# Cross-asset appendix
# ----------------------------
def appendix_cross_asset(frames: Dict[str, pd.DataFrame]) -> List[plt.Figure]:
    figs: List[plt.Figure] = []

    ret_map = {}
    for name, df in frames.items():
        r = _get_returns(df, name)
        if not r.empty:
            ret_map[name] = r

    if ret_map:
        rets = pd.DataFrame(ret_map).sort_index()
        fig, ax = plt.subplots()
        sns.heatmap(rets.corr(), annot=True, cmap="viridis", ax=ax)
        ax.set_title("Appendix • Cross‑Asset Daily Log Return Correlation")
        figs.append(fig)

    return figs

# ----------------------------
# PDF orchestration
# ----------------------------
def build_garch_volatility_report(
    frames: Dict[str, pd.DataFrame],
    pdf_title: str = "Univariate_GARCH_Volatility_Report.pdf",
    use_student_t: bool = False,
    forecast_h: int = DEFAULT_FORECAST_H,
    show_plots: bool = False
) -> None:
    dist = "t" if use_student_t else "normal"
    ordered = sorted(frames.items(), key=lambda kv: kv[0])

    figs: List[plt.Figure] = []

    settings = {
        "Model": "GARCH(1,1) with mean=constant; ML estimation",
        "Innovation Distribution": "Student‑t" if use_student_t else "Gaussian",
        "Forecast Horizon (h)": forecast_h,
        "Volatility Units": "Annualized % (daily σ * √252 * 100)",
        "Realized Vol Window": f"{REALIZED_VOL_WINDOW_DAYS} trading days (annualized %)",
        "ACF Shaded Band": f"{int((1-ACF_ALPHA)*100)}% bounds around 0 under H₀ (no autocorrelation)",
        "Assets Included": ", ".join([k for k, _ in ordered]),
    }
    figs.append(_table_figure_from_dict("Report Settings", settings))

    for name, df in ordered:
        if df is None or df.empty:
            figs.append(_table_figure_from_dict(
                f"{_safe_title(name)} • Data Warning",
                {"Message": "Empty DataFrame provided for this asset."}
            ))
            continue
        figs.extend(garch_diagnostics_for_asset(name, df, dist=dist, forecast_h=forecast_h))

    figs.extend(appendix_cross_asset({k: v for k, v in ordered if v is not None and not v.empty}))

    with PdfPages(pdf_title) as pdf:
        cover, ax = plt.subplots()
        ax.axis("off")
        ax.text(0.02, 0.90, "Univariate GARCH Volatility & Diagnostics Report",
                fontsize=20, weight="bold")
        ax.text(0.02, 0.82, "Assets: " + ", ".join([k for k, _ in ordered]), fontsize=12)
        ax.text(0.02, 0.76,
                "This report estimates a GARCH(1,1) per asset, visualizes annualized conditional volatility, "
                "overlays realized volatility (100D), checks residual diagnostics (including ACF with significance bands), "
                "and produces closed‑form h‑step forecasts (shown as annualized volatility).",
                fontsize=11)
        pdf.savefig(cover)
        plt.close(cover)

        for fig in figs:
            pdf.savefig(fig)
            plt.close(fig)

    print(f"✅ PDF saved: {pdf_title}")

    if show_plots:
        for fig in figs:
            fig.show()

# ----------------------------
# Example usage
# ----------------------------
if __name__ == "__main__":
    frames = {
        "SPY": spy_df,
        "GLD": gld_df,
        "CL=F": clf_df,
        "PSX": psx_df,
        "VLO": vlo_df,
        "MPC": mpc_df,
        "EUR_USD": eurusd_df,
    }

    build_garch_volatility_report(
        frames,
        pdf_title="Univariate_GARCH_Volatility_Report.pdf",
        use_student_t=False,
        forecast_h=90,
        show_plots=False
    )

/tmp/ipykernel_11276/3182991058.py:601: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()
/tmp/ipykernel_11276/3182991058.py:754: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  pdf.savefig(cover)


✅ PDF saved: Univariate_GARCH_Volatility_Report.pdf
